<a href="https://colab.research.google.com/github/tlsdmswn01/Text_mining_project/blob/main/%EA%B8%B0%EC%97%85%20%EC%A0%95%EB%B3%B4%20%EC%A0%9C%EA%B3%B5-%EA%B5%B0%EC%A7%91%ED%99%94%2C%20%EC%9C%A0%EC%82%AC%EB%8F%84%20%ED%99%9C%EC%9A%A9/%EC%A0%84%EC%B2%B4%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 형태소 분석
!pip install konlpy
# 연관 키워드 추출
!pip install keybert

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import re
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE

from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from keybert import KeyBERT

from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

### 뉴스 기사 불러오기
- 빅카인즈에서 데이터 저장(https://www.bigkinds.or.kr/v2/news/index.do)

In [ ]:
df = pd.read_excel('/content/drive/Shareddrives/텍스트 마이닝 프로젝트/웹 크롤링 코드 및 데이터/삼성전자_주가_2020_2023.xlsx')

df['일자'] = df['일자'].astype('int')
df = df[df['일자'] >= 20230501] # 2023년 5월 이후의 데이터만 사용(최근 뉴스 제공 위해)

df = df[['일자', '언론사', '제목', '본문', '키워드' ,'URL']]

# 제목에서 [], () 안에 있는 내용 지우기
df['제목'] = df['제목'].apply(lambda x : re.sub(r'\[[^]]*\]', '', x))
df['제목'] = df['제목'].apply(lambda x : re.sub(r'\([^)]*\)', '', x))

# 중복 행 제거
df = df.drop_duplicates(subset='제목', keep='first')
#df = df.dropna(inplace=True)

In [ ]:
df.columns

### 뉴스 기사 군집화
- 비슷한 내용의 다른 기사가 존재하는 것을 확인함
- 군집화를 통해 비슷한 내용이면 하나의 기사만 출력

In [ ]:
# RANKS NL에서 제공하는 한국어 불용어 사전 활용

stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()
stopwords = sum(stopwords, [])

with open('nsmc_stopwords.txt', 'wt') as f:
    f.write('\n'.join(stopwords))  # 리스트 -> 문자열

In [ ]:
def preprocessing(review):
    okt = Okt()

    f = open('nsmc_stopwords.txt')
    stop_words = f.read().split()

    # 1. 한글, 숫자, 공백을 제외한 문자 모두 제거.
    review_text = re.sub("[^가-힣\s0-9]", "", review)

    # 2. okt 객체를 활용해서 형태소 토큰화 + 품사 태깅 + stemming
    word_review = okt.pos(review_text, stem=True)

    # 노이즈 & 불용어 제거
    word_review = [(token, pos) for token, pos in word_review if not token in stop_words and len(token) > 1]

    # 명사, 동사, 형용사 추출
    word_review = [token for token, pos in word_review if pos in ['Noun', 'Verb', 'Adjective']]

    return word_review

In [ ]:
text = df['제목']

In [ ]:
text = list(map(lambda x : preprocessing(x), text))

In [ ]:
def dummyTokenizer(x):
    return x

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TfidfVectorizer 객체 생성
tfidf_vectorizer = TfidfVectorizer(tokenizer = dummyTokenizer, preprocessor = dummyTokenizer, max_features=10000)

# 텍스트 데이터를 TF-IDF 벡터로 변환
tfidf = tfidf_vectorizer.fit_transform(text)

# TF-IDF 벡터화된 결과를 배열로 변환
tfidf_array = tfidf.toarray()

# 단어 목록 확인
feature_names_tfidf = tfidf_vectorizer.get_feature_names_out()

# TF-IDF 벡터화된 결과 및 단어 목록 출력
print("TfidfVectorized Data:")
print(tfidf_array)
print("Feature Names:")
print(feature_names_tfidf)

In [ ]:
eps_values = [0.3, 0.5, 0.7]
min_samples_values = [1, 3, 5]

# 그래프를 그릴 격자 생성
fig, axes = plt.subplots(len(eps_values), len(min_samples_values), figsize=(15, 10))

for i, eps in enumerate(eps_values):
    for j, min_samples in enumerate(min_samples_values):
        # DBSCAN 모델 생성
        model = DBSCAN(eps=eps, min_samples=min_samples, metric="cosine")
        result = model.fit_predict(tfidf_array)

        # t-SNE를 사용하여 데이터를 2차원으로 축소
        tsne = TSNE(n_components=2, random_state=1004, perplexity=20)
        X_tsne = tsne.fit_transform(tfidf_array)

        # 군집화 결과와 함께 서브플롯에 그래프 그리기
        axes[i, j].scatter(X_tsne[:, 0], X_tsne[:, 1], c=result, cmap='jet')
        axes[i, j].set_title(f'eps={eps}, min_samples={min_samples}')
        axes[i, j].axis('off')  # 축 제거

# 그래프 출력
plt.tight_layout()
plt.show()


In [ ]:
# DBSCAN Clustering

from sklearn.cluster import DBSCAN
import numpy as np

model = DBSCAN(eps=0.5,min_samples=5, metric = "cosine")
result = model.fit_predict(tfidf_array)

cluster_labels = result

In [ ]:
df['cluster1st'] = result

In [ ]:
print('군집개수 :', result.max())

In [ ]:
clusters = []

for cluster_num in set(result):
    # -1,0은 노이즈 판별이 났거나 클러스터링이 안된 경우
    if(cluster_num == -1 or cluster_num == 0):
        continue
    else:
        print("cluster num : {}".format(cluster_num))
        temp_df = df[df['cluster1st'] == cluster_num] # cluster num 별로 조회
        clusters.append(cluster_num)
        for title in temp_df['제목']:
            print(title)
        print()

In [ ]:
clu_df = df[(df['cluster1st'] != 0) & (df['cluster1st'] != -1)]

In [ ]:
len(clu_df['cluster1st'].unique())

In [ ]:
df_list = []
n = len(clu_df['cluster1st'].unique())
for i in range(1, n + 1):
    df_list.append(clu_df[clu_df['cluster1st'] == i])

In [ ]:
df_list[0]

### 코사인 유사도(메인 기사)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# 각 데이터프레임에 대해 정리
for i in range(len(df_list)):
    dfn = df_list[i]
    dfnk = dfn['키워드']

    # TF-IDF 벡터화 객체 생성 및 학습
    tfidf_vec = TfidfVectorizer(stop_words = 'english')
    vec = tfidf_vec.fit_transform(dfnk)

    best = []

    # 각 행에 대한 코사인 유사도 계산
    for k in range(dfnk.count()):
        all_cos_sim = cosine_similarity(vec[k],vec)
        all_cos_sim = np.delete(all_cos_sim, k, axis=1)
        sum_1 = np.sum(all_cos_sim)
        best.append(sum_1)

    # 'sim'열에 유사도 합계 추가
    df_list[i]['sim'] = best

In [ ]:
title = []  # '제목' 정보를 저장할 리스트
url = []    # 'URL' 정보를 저장할 리스트
doc = []    # '키워드' 정보를 저장할 리스트

# 각 데이터프레임에 대해 처리
for i in range(len(df_list)):
    df_result = df_list[i].sort_values(by='sim',ascending=False)
    top_entry = df_result.iloc[0, :] # 유사도가 높은 항목 선택

    print(top_entry['제목'])
    print(top_entry['URL'])
    print('==============================================')

    title.append(df_result.iloc[0,:]['제목'])
    doc.append(df_result.iloc[0,:]['키워드'])
    url.append(df_result.iloc[0,:]['URL'])

In [ ]:
# 집단내에서 cosine 유사도가 가장 높은 기사 하나만 선택 -> dataframe형태
main_report=pd.DataFrame({'title':title,'key':doc,'url':url})
main_report

### 자카드 유사도(유사 사업 기업)

In [ ]:
import pickle

with open("/content/drive/Shareddrives/텍스트 마이닝 프로젝트/웹 크롤링 코드 및 데이터/삼성전자 사업개요.pkl","rb") as f:
    samsung_load = pickle.load(f)
samsung_load.append('삼성')
samsung_load.append('전자')
samsung_load.append('삼성전자')

In [ ]:
samsung_load

In [ ]:
samsung_load.count('삼성')

In [ ]:
def jaccard_sim(d1,d2):
    s1=set(d1)
    s2=set(d2)
    return float(len(s1.intersection(s2))/len(s1.union(s2)))


In [ ]:
dfnk=main_report['key']
jaccard_sim(dfnk.iloc[0].split(','),samsung_load)

In [ ]:
best=[]
for k in range(dfnk.count()):
    best.append(jaccard_sim(dfnk.iloc[k].split(','),samsung_load))
main_report['sim']=best

In [ ]:
main_report=main_report.sort_values(by='sim',ascending=False)
main_report.loc[:10,['url','key']]

In [ ]:
main_report.to_csv('메인기사 10개 추출.csv')

### 연관 키워드 추출

In [ ]:
def BERT(title, num):
    # 특정 군집 번호에 해당하는 '키워드' 열 데이터 추출
    array_text = pd.DataFrame(df[df['cluster1st'] == num]['키워드']).to_numpy()

    bow = []
    from keybert import KeyBERT
    kw_extractor = KeyBERT('distilbert-base-nli-mean-tokens')

    # 각 문장에 대한 키워드 추출
    for j in range(len(array_text)):
        keywords = kw_extractor.extract_keywords(array_text[j][0])
        bow.append(keywords)

    new_bow = []
    for i in range(0, len(bow)):
        for j in range(len(bow[i])):
            new_bow.append(bow[i][j])

    keyword = pd.DataFrame(new_bow, columns=['keyword', 'weight'])
    keyword = keyword.groupby('keyword').agg('sum').sort_values('weight', ascending=False).head(20)
    return keyword

In [ ]:
keyword_df = pd.DataFrame()
cluster_list = []

for i in range(1, 15):  # 군집 -1, 0은 오분류, 이상치로 판단
    cluster_df = BERT('삼성전자', i)
    cluster_list.append(cluster_df)

# index 맞추기 위해
for i in range(len(cluster_list)):
    cluster_list[i] = cluster_list[i].reset_index()

# 각 군집의 결과를 하나의 데이터프레임으로 합치고 가중치 순으로 정렬
weight_df = pd.concat(cluster_list, axis=0)
weight_df = weight_df.sort_values(by='weight', ascending=False)

#weight_df.to_csv('/content/drive/Shareddrives/텍스트 마이닝 프로젝트/웹 크롤링 코드 및 데이터/weight_df.csv', index=False)

# '삼성전자' 키워드 제외
weight_df = weight_df[weight_df['keyword'] != '삼성전자']
weight_df